In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://slidetodoc.com/presentation_image/53cdb36ec5d2f9c3437ea4791f425e03/image-13.jpg)SlideToDoc.com

In [ ]:
# Get the unified BCG Strain spreadsheet
df = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2022/data.csv', delimiter=',')
pd.set_option('display.max_columns', None)
df.head()

#All script by Aleksey Bilogur https://www.kaggle.com/code/residentmario/notes-on-semi-supervised-learning/notebook

In [ ]:
#Code by Aleksey Bilogur https://www.kaggle.com/code/residentmario/notes-on-semi-supervised-learning/notebook

import seaborn as sns
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=5000, n_features=2, n_informative=2,
                           n_redundant=0, n_repeated=0, n_classes=3,
                           n_clusters_per_class=1,
                           weights=[0.01, 0.05, 0.94],
                           class_sep=0.8, random_state=0)

import matplotlib.pyplot as plt
colors = ['#ef8a62' if v == 0 else '#f7f7f7' if v == 1 else '#67a9cf' for v in y]
kwarg_params = {'linewidth': 1, 'edgecolor': 'black'}
fig = plt.Figure(figsize=(12,6))
plt.scatter(X[:, 0], X[:, 1], c=colors, **kwarg_params);

In [ ]:
#Code by Aleksey Bilogur https://www.kaggle.com/code/residentmario/notes-on-semi-supervised-learning/notebook

from sklearn.semi_supervised import LabelPropagation
from sklearn.datasets import make_classification


def label(df, class_sep, trans=LabelPropagation()):
    """Label a point cloud with the given class separation """
    X, y = make_classification(n_samples=5000, n_features=2, n_informative=2,
                               n_redundant=0, n_repeated=0, n_classes=3,
                               n_clusters_per_class=1,
                               weights=[0.05, 0.10, 0.85],
                               class_sep=class_sep, random_state=0)
    
    X_l = X.shape[0]
    np.random.seed(42)
    unl_idx = np.random.randint(0, len(X), size=X_l - 500)
    l_idx = list(set(range(X_l)).difference(set(unl_idx)))
    X_unlabeled, X_labeled = X[unl_idx], X[l_idx]
    y_unlabeled, y_labeled = y[unl_idx], y[l_idx]
    
    trans = LabelPropagation()
    trans.fit(X_labeled, y_labeled)

    y_unlabeled_pred = trans.predict(X_unlabeled)
    
    r = (pd.DataFrame({'y': y_unlabeled, 'y_pred': y_unlabeled_pred}))
    
    return X_unlabeled, X_labeled, y_unlabeled, y_labeled, r

In [ ]:
#Code by Aleksey Bilogur https://www.kaggle.com/code/residentmario/notes-on-semi-supervised-learning/notebook

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

cmap = {0: '#ef8a62',1: '#f7f7f7', 2: '#67a9cf'}
fig, axarr = plt.subplots(2, 4, figsize=(12, 6))

for plot_n, sep in enumerate([0.1, 0.5, 1, 2]):
    X_unlabeled, X_labeled, y_unlabeled, y_labeled, r = label(X, sep, trans=LabelPropagation())

    for classnum, color in cmap.items():
        k_idx_labeled = np.where(y_labeled == classnum)[0]
        X_v, y_v = X_labeled[k_idx_labeled], y_labeled[k_idx_labeled]
        axarr[0][plot_n].plot(X_v[:, 0], X_v[:, 1], marker='o', linewidth=0, alpha=0.25, c=color)
        
        k_idx_unlabeled_misclassified = np.where(r.query("y == @classnum & y != y_pred").values)[0]
        X_v, y_v = X_unlabeled[k_idx_unlabeled_misclassified], y_unlabeled[k_idx_unlabeled_misclassified]
        axarr[0][plot_n].plot(X_v[:, 0], X_v[:, 1], marker='o', 
                              markeredgewidth=1, markeredgecolor='black', linewidth=0, c='None', zorder=10)
        
        result = r.rename_axis(None).groupby('y').apply(lambda df: (df.y == df.y_pred).sum() / len(df))
        result.plot.bar(color=cmap.values(), ax=axarr[1][plot_n])
        
plt.suptitle("$LabelPropagation$ Performance Benchmark, $Sep \in [0.1, 0.5, 1, 2]$");

In [ ]:
#Code by Aleksey Bilogur https://www.kaggle.com/code/residentmario/notes-on-semi-supervised-learning/notebook

from sklearn.semi_supervised import LabelSpreading

fig, axarr = plt.subplots(2, 4, figsize=(12, 6))

for plot_n, sep in enumerate([0.1, 0.5, 1, 2]):
    X_unlabeled, X_labeled, y_unlabeled, y_labeled, r = label(X, sep, trans=LabelSpreading())

    for classnum, color in cmap.items():
        k_idx_labeled = np.where(y_labeled == classnum)[0]
        X_v, y_v = X_labeled[k_idx_labeled], y_labeled[k_idx_labeled]
        axarr[0][plot_n].plot(X_v[:, 0], X_v[:, 1], marker='o', linewidth=0, alpha=0.25, c=color)
        
        k_idx_unlabeled_misclassified = np.where(r.query("y == @classnum & y != y_pred").values)[0]
        X_v, y_v = X_unlabeled[k_idx_unlabeled_misclassified], y_unlabeled[k_idx_unlabeled_misclassified]
        axarr[0][plot_n].plot(X_v[:, 0], X_v[:, 1], marker='o', 
                              markeredgewidth=1, markeredgecolor='black', linewidth=0, c='None', zorder=10)
        
        result = r.rename_axis(None).groupby('y').apply(lambda df: (df.y == df.y_pred).sum() / len(df))
        result.plot.bar(color=cmap.values(), ax=axarr[1][plot_n])
        
plt.suptitle("$LabelSpreading$ Performance Benchmark, $Sep \in [0.1, 0.5, 1, 2]$");

In [ ]:
df.isnull().sum()

#Application benchmark

"Aleksey tried to apply this algorithm to a real dataset, the Open Powerlifting Database, to assign Division information to fields lacking it. However, this turned out to be foolish, as the class clusters were not sufficiently well-distributed. The accuracy was nearly 0."

#I changed Division to F_2_0 and the columns too. Obviously, it didn't work.

In [ ]:
#Code by Aleksey Bilogur https://www.kaggle.com/code/residentmario/notes-on-semi-supervised-learning/notebook

import pandas as pd
df = pd.read_csv("../input/tabular-playground-series-jun-2022/data.csv")

# import missingno as msno
# msno.matrix(df.sample(1000))

import numpy as np
np.random.seed(42)

cols = ['F_2_0', 'F_2_1', 'F_2_2', 'F_2_3', 'F_2_4', 'F_2_5', 'F_2_6', 'F_2_7', 'F_2_8', 'F_2_9', 'F_2_10', 'F_2_11', 'F_2_12', 'F_2_13', 'F_2_14', 'F_2_15', 'F_2_16', 'F_2_17', 'F_2_18', 'F_2_19', 'F_2_20', 'F_2_21', 'F_2_22', 'F_2_23', 'F_2_24']

df_nonempty = (df
   .dropna(subset=['F_2_0'])
   #.drop(['MeetID', 'Name', 'Place', 'WeightClassKg'], axis='columns')
   .pipe(lambda df: df.assign(
        # Sex=df.Sex.astype('category').cat.codes,
         F_2_0=df.F_2_0.astype('uint8').int.codes,
         #Equipment=df.Equipment.astype('category').cat.codes
   ))
   .pipe(lambda df: df[df.F_2_0.isin(df.F_2_0.value_counts().head(10).index.values)])
  # .loc[:, cols]
  # .dropna()
  # .reset_index(drop=True)               
)
unl_idx = np.random.randint(0, len(df_nonempty), size=len(df_nonempty) - 500)
l_idx = list(set(range(len(df_nonempty))).difference(set(unl_idx)))

#cols = [c for c in cols if c != 'Division']

#X_unlabeled, X_labeled = df_nonempty.loc[unl_idx, cols].head(5000), df_nonempty.loc[l_idx, cols].head(5000)
#y_unlabeled, y_labeled = df_nonempty.loc[unl_idx, 'Division'].head(5000), df_nonempty.loc[l_idx, 'Division'].head(5000)

In [ ]:
#Code by Aleksey Bilogur https://www.kaggle.com/code/residentmario/notes-on-semi-supervised-learning/notebook

from sklearn.semi_supervised import LabelPropagation

trans = LabelPropagation()
trans.fit(X_labeled, y_labeled)

np.random.seed(42)
y_unlabeled_pred = trans.predict(X_unlabeled)

print("Performance on division imputation: {0}% Accuracy".format((y_unlabeled_pred == y_unlabeled)\
                                                                 .pipe(lambda df: df.sum() / len(df))))

#Another script that I screwed and can't fix it.

#Conclusion

"Semi-supervised learning is a restatement of the missing data imputation problem which is specific to the small-sample, missing-label case. This problem gets its own name likely because it is so commonly encountered in research and dataset generation contexts. It's a useful tool to know about more generally for missing data imputation from a limited sample size, but the algorithms have poor performance characteristics for larger samples. In those cases, perhaps try applying machine learning to the problem directly."

By Aleksey Bilogur https://www.kaggle.com/code/residentmario/notes-on-semi-supervised-learning/notebook

Acknowledgement:

Aleksey Bilogur https://www.kaggle.com/code/residentmario/notes-on-semi-supervised-learning/notebook